In [9]:
import threading

In [10]:
import time

In [ ]:
import sys
import threading
import logging
from time import sleep

class StoppableThread(threading.Thread):
   """
   Implements a thread that can be stopped.
   """
   def __init__(self,  name, target, args=()):
      super(StoppableThread, self).__init__(name=name, target=target, args=args)
      self._status = 'running'
   def stop_me(self):
      if (self._status == 'running'):
         self._status = 'stopping'
   def stopped(self):
      self._status = 'stopped'
   def is_running(self):
      return (self._status == 'running')
   def is_stopping(self):
      return (self._status == 'stopping')
   def is_stopped(self):
      return (self._status == 'stopped')

dThreads = {}

def ThreadFunc(id, arg1, arg2):
   thread = dThreads[id]
   while (thread.is_running()):
      logging.info('Tick...')
      sleep(4)
   thread.stopped()

def StartThread(id, arg1, arg2):
   """
   Starts a thread and adds an entry to the global dThreads dictionary.
   """
   logging.info('Starting %s' % id)
   dThreads[id] = StoppableThread(name=id, target=ThreadFunc,
                                  args=(id,arg1,arg2))
   dThreads[id].start()

def StopThread(id):
   """
   Stops a thread and removes its entry from the global dThreads dictionary.
   """
   thread = dThreads[id]
   if (thread.is_running()):
      logging.info('Stopping %s' % id)
      thread.stop_me()
      thread.join()
      logging.info('Stopped %s' % id)
      del dThreads[id]

if __name__ == "__main__":
   logging.basicConfig(format='%(threadName)s:%(asctime)s:%(levelname)s:%(message)s',
                       stream=sys.stdout, level=logging.INFO)


   StartThread('one', 'argval1', 'argval2')
   sleep(2)
   StartThread('two', 'argval3', 'argval4')
   sleep(10)
   StopThread('one')
   sleep(10)
   StopThread('two')

In [35]:
import functools
def rlocker(func):
    '''wraps func in a reentrant lock'''
    rlock = threading.RLock()
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        with rlock:
            ret = func(*args, **kwargs)
        return ret
    return wrapper

In [79]:
lockingPrinter = rlocker(print)

def doSomething(msg, val):
    lockingPrinter(msg, ': ', val)



def mirrorRange(rg):
    c = list(rg).copy()
    c.extend(rg[-2:0:-1])
    return c

def cycle(fn, rg, duration, repeat=False, stopEvent = None):
    wait = duration / len(rg)
    cont = True
    while cont:
        if stopEvent:
            if stopEvent.isSet():
                break
        for v in rg:
            fn(v)
            time.sleep(wait)
        cont = repeat

        
        
def cycle2(fn, args, durs, stopEvent = None):
    ''' 
    args and durs are generators, they may end or may 
    go on forever (in which case the stopEvent can end things)
    '''
    while not(stopEvent and stopEvent.isSet()):
        v = next(args)
        d = next(args)
        if v == None or d == None:
            break
        fn(v)
        time.sleep(d)
        
def pc(msg, num, dur, repeat, stopper):
    cycle(lambda val: doSomething(msg, val), mirrorRange(range(num)), dur, repeat, stopper)
    
def pc2(msg, num, dur, stopper):
    cycle2(lambda val: doSomething(msg, val), mirrorRange(range(num)), dur, stopper)

In [80]:
def numGen(n):
    while True:
        yield n
    

In [81]:
g = numGen(5)

In [82]:
next(g)

5

In [83]:
s2 = threading.Event()
t2  = threading.Thread(target = pc2, args=("msg", numGen(5), numGen(0.5),  s))

In [84]:
t2.start()

Exception in thread Thread-40:
Traceback (most recent call last):
  File "C:\Users\g.stalker-wilde\AppData\Local\conda\conda\envs\tensorflow\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\Users\g.stalker-wilde\AppData\Local\conda\conda\envs\tensorflow\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-79-296b312631eb>", line 44, in pc2
    cycle2(lambda val: doSomething(msg, val), mirrorRange(range(num)), dur, stopper)
TypeError: 'generator' object cannot be interpreted as an integer



In [38]:
s2.set()

In [39]:
s = threading.Event()
t  = threading.Thread(target = pc, args=("msg", 17, 2, True, s))

In [ ]:
t.start()

In [ ]:
s.set()

In [51]:
stopEvent1 = threading.Event()
stopEvent3 = threading.Event()
args = [('one', 5, 3, True, stopEvent1), ('two', 10, 5, False, None), ('three', 7, 7, True, stopEvent3)]

In [52]:
ts = [threading.Thread(target=pc, args = (m,n,d, r,s)) for (m,n,d,r,s) in args]

In [53]:
[t.start() for t in ts]

one :  0
two :  0
three :  0


[None, None, None]

two :  1
two :  2
three :  1
two :  3
two :  4
three :  2
one :  1
two :  5
two :  6
three :  3
two :  7
two :  8
three :  4
one :  2
two :  9
two :  8
three :  5
two :  7
two :  6
three :  6
two :  5
one :  3
two :  4
three :  5
two :  3
two :  2
three :  4
two :  1
one :  4
three :  3
three :  2
one :  3
three :  1
three :  0


In [54]:
stopEvent1.set()

one :  2
three :  1


In [55]:
stopEvent3.set()

three :  2
one :  1
three :  3
three :  4
three :  5
three :  6
three :  5
three :  4
three :  3
three :  2
three :  1
